In [185]:
import torch
import torch.nn as nn
import pandas as pd
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, random_split
import numpy as np

Perform operations on GPU

In [186]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


Load data

In [187]:
dataframe_raw = pd.read_csv('data.csv', delimiter=",")

Explore data

In [188]:
dataframe_raw.head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


In [189]:
dataframe_raw = dataframe_raw.drop(['dteday', 'instant', 'casual', 'registered'], axis=1)
dataframe_raw.head()

,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,cnt
0,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,16
1,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,40
2,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,32
3,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,13
4,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,1


In [190]:
dataframe_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   season      10886 non-null  int64  
 1   yr          10886 non-null  int64  
 2   mnth        10886 non-null  int64  
 3   hr          10886 non-null  int64  
 4   holiday     10886 non-null  int64  
 5   weekday     10886 non-null  int64  
 6   workingday  10886 non-null  int64  
 7   weathersit  10886 non-null  int64  
 8   temp        10886 non-null  float64
 9   atemp       10886 non-null  float64
 10  hum         10886 non-null  float64
 11  windspeed   10886 non-null  float64
 12  cnt         10886 non-null  int64  
dtypes: float64(4), int64(9)
memory usage: 1.1 MB


In [191]:
dataframe_raw.describe()

,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,cnt
count,10886.000000,10886.000000,10886.000000,10886.000000,10886.000000,10886.000000,10886.000000,10886.000000,10886.000000,10886.000000,10886.000000,10886.000000,10886.000000
mean,2.506614,0.501929,6.521495,11.541613,0.028569,2.998622,0.680875,1.418427,0.493436,0.473102,0.618865,0.191036,191.574132
std,1.116174,0.500019,3.444373,6.915838,0.166599,2.007770,0.466159,0.633839,0.190039,0.169492,0.192450,0.121859,181.144454
min,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.020000,0.015200,0.000000,0.000000,1.000000
25%,2.000000,0.000000,4.000000,6.000000,0.000000,1.000000,0.000000,1.000000,0.340000,0.333300,0.470000,0.104500,42.000000
50%,3.000000,1.000000,7.000000,12.000000,0.000000,3.000000,1.000000,1.000000,0.500000,0.484800,0.620000,0.194000,145.000000
75%,4.000000,1.000000,10.000000,18.000000,0.000000,5.000000,1.000000,2.000000,0.640000,0.621200,0.770000,0.253700,284.000000
max,4.000000,1.000000,12.000000,23.000000,1.000000,6.000000,1.000000,4.000000,1.000000,0.909100,1.000000,0.850700,977.000000


In [192]:
dataframe_raw.describe()
dataframe_raw.corr()

,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,cnt
season,1.000000,-0.004797,0.971524,-0.006546,0.029368,-0.009691,-0.008126,0.008879,0.258689,0.264744,0.190610,-0.147121,0.163439
yr,-0.004797,1.000000,-0.004932,-0.004234,0.012021,-0.018162,-0.002482,-0.012548,0.061226,0.058540,-0.078606,-0.015221,0.260403
mnth,0.971524,-0.004932,1.000000,-0.006818,0.001731,-0.001012,-0.003394,0.012144,0.257589,0.264173,0.204537,-0.150192,0.166862
hr,-0.006546,-0.004234,-0.006818,1.000000,-0.000354,-0.001660,0.002780,-0.022740,0.145430,0.140343,-0.278011,0.146631,0.400601
holiday,0.029368,0.012021,0.001731,-0.000354,1.000000,-0.104800,-0.250491,-0.007074,0.000295,-0.005215,0.001929,0.008409,-0.005393
weekday,-0.009691,-0.018162,-0.001012,-0.001660,-0.104800,1.000000,0.035554,-0.016439,-0.026143,-0.035079,-0.059722,-0.006178,0.027690
workingday,-0.008126,-0.002482,-0.003394,0.002780,-0.250491,0.035554,1.000000,0.033772,0.029966,0.024660,-0.010880,0.013373,0.011594
weathersit,0.008879,-0.012548,0.012144,-0.022740,-0.007074,-0.016439,0.033772,1.000000,-0.055035,-0.055376,0.406244,0.007261,-0.128655
temp,0.258689,0.061226,0.257589,0.145430,0.000295,-0.026143,0.029966,-0.055035,1.000000,0.984948,-0.064949,-0.017852,0.394454
atemp,0.264744,0.058540,0.264173,0.140343,-0.005215,-0.035079,0.024660,-0.055376,0.984948,1.000000,-0.043536,-0.057473,0.389784


In [193]:
dataframe_raw.corr()

,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,cnt
season,1.000000,-0.004797,0.971524,-0.006546,0.029368,-0.009691,-0.008126,0.008879,0.258689,0.264744,0.190610,-0.147121,0.163439
yr,-0.004797,1.000000,-0.004932,-0.004234,0.012021,-0.018162,-0.002482,-0.012548,0.061226,0.058540,-0.078606,-0.015221,0.260403
mnth,0.971524,-0.004932,1.000000,-0.006818,0.001731,-0.001012,-0.003394,0.012144,0.257589,0.264173,0.204537,-0.150192,0.166862
hr,-0.006546,-0.004234,-0.006818,1.000000,-0.000354,-0.001660,0.002780,-0.022740,0.145430,0.140343,-0.278011,0.146631,0.400601
holiday,0.029368,0.012021,0.001731,-0.000354,1.000000,-0.104800,-0.250491,-0.007074,0.000295,-0.005215,0.001929,0.008409,-0.005393
weekday,-0.009691,-0.018162,-0.001012,-0.001660,-0.104800,1.000000,0.035554,-0.016439,-0.026143,-0.035079,-0.059722,-0.006178,0.027690
workingday,-0.008126,-0.002482,-0.003394,0.002780,-0.250491,0.035554,1.000000,0.033772,0.029966,0.024660,-0.010880,0.013373,0.011594
weathersit,0.008879,-0.012548,0.012144,-0.022740,-0.007074,-0.016439,0.033772,1.000000,-0.055035,-0.055376,0.406244,0.007261,-0.128655
temp,0.258689,0.061226,0.257589,0.145430,0.000295,-0.026143,0.029966,-0.055035,1.000000,0.984948,-0.064949,-0.017852,0.394454
atemp,0.264744,0.058540,0.264173,0.140343,-0.005215,-0.035079,0.024660,-0.055376,0.984948,1.000000,-0.043536,-0.057473,0.389784


Prepare dataset for training

In [194]:
input_cols = list(dataframe_raw.columns)[:-1]
output_cols = ['cnt']
input_cols, output_cols

(['season',
  'yr',
  'mnth',
  'hr',
  'holiday',
  'weekday',
  'workingday',
  'weathersit',
  'temp',
  'atemp',
  'hum',
  'windspeed'],
 ['cnt'])

In [195]:
def dataframe_to_arrays(dataframe):
    # Extract input & outputs as numpy arrays
    inputs_array = dataframe[input_cols].to_numpy()
    targets_array = dataframe[output_cols].to_numpy()
    return inputs_array, targets_array

In [196]:
inputs_array, targets_array = dataframe_to_arrays(dataframe_raw)
inputs_array, targets_array

(array([[ 1.    ,  0.    ,  1.    , ...,  0.2879,  0.81  ,  0.    ],
        [ 1.    ,  0.    ,  1.    , ...,  0.2727,  0.8   ,  0.    ],
        [ 1.    ,  0.    ,  1.    , ...,  0.2727,  0.8   ,  0.    ],
        ...,
        [ 4.    ,  1.    , 12.    , ...,  0.3182,  0.61  ,  0.2239],
        [ 4.    ,  1.    , 12.    , ...,  0.3485,  0.61  ,  0.0896],
        [ 4.    ,  1.    , 12.    , ...,  0.3333,  0.66  ,  0.1343]]),
 array([[ 16],
        [ 40],
        [ 32],
        ...,
        [168],
        [129],
        [ 88]], dtype=int64))

Convert numpy arrays to torch tensors

In [197]:
inputs = torch.from_numpy(inputs_array).type(torch.float32)
targets = torch.from_numpy(targets_array).type(torch.float32)
inputs, targets


(tensor([[ 1.0000,  0.0000,  1.0000,  ...,  0.2879,  0.8100,  0.0000],
         [ 1.0000,  0.0000,  1.0000,  ...,  0.2727,  0.8000,  0.0000],
         [ 1.0000,  0.0000,  1.0000,  ...,  0.2727,  0.8000,  0.0000],
         ...,
         [ 4.0000,  1.0000, 12.0000,  ...,  0.3182,  0.6100,  0.2239],
         [ 4.0000,  1.0000, 12.0000,  ...,  0.3485,  0.6100,  0.0896],
         [ 4.0000,  1.0000, 12.0000,  ...,  0.3333,  0.6600,  0.1343]]),
 tensor([[ 16.],
         [ 40.],
         [ 32.],
         ...,
         [168.],
         [129.],
         [ 88.]]))

In [198]:
print(f'Shape of input tensor and target tensor:    {inputs.shape} | {targets.shape} ')
print(f'Datatype of input tensor and target tensor: {inputs.dtype} | {targets.dtype} ')


Shape of input tensor and target tensor:    torch.Size([10886, 12]) | torch.Size([10886, 1]) 
Datatype of input tensor and target tensor: torch.float32 | torch.float32 


Convert torch tensor to dataset


In [199]:
dataset = TensorDataset(inputs, targets)
dataset

Split dataset into 2 parts - training set, validation set

In [200]:
train_ds, val_ds = random_split(dataset, [8460, 2426])
batch_size = 20

train_loader = DataLoader(train_ds, batch_size, shuffle=True)
val_loader = DataLoader(val_ds, batch_size)

Display first data batch from train set

In [201]:
for xb, yb in train_loader:
    print(f'inputs: {xb}')
    print(f'targets: {yb}')
    break


inputs: tensor([[ 2.0000,  0.0000,  5.0000,  6.0000,  0.0000,  3.0000,  1.0000,  3.0000,
          0.3400,  0.3030,  0.9300,  0.3881],
        [ 1.0000,  1.0000,  3.0000, 18.0000,  0.0000,  0.0000,  0.0000,  1.0000,
          0.5200,  0.5000,  0.2900,  0.2836],
        [ 3.0000,  0.0000,  9.0000, 16.0000,  0.0000,  4.0000,  1.0000,  3.0000,
          0.6400,  0.5606,  0.9400,  0.2836],
        [ 1.0000,  0.0000,  1.0000, 22.0000,  0.0000,  1.0000,  1.0000,  1.0000,
          0.1400,  0.1515,  0.6900,  0.1343],
        [ 1.0000,  0.0000,  2.0000, 18.0000,  0.0000,  3.0000,  1.0000,  1.0000,
          0.4000,  0.4091,  0.4000,  0.2239],
        [ 4.0000,  1.0000, 11.0000, 23.0000,  0.0000,  0.0000,  0.0000,  2.0000,
          0.3000,  0.3030,  0.5600,  0.1642],
        [ 3.0000,  1.0000,  7.0000,  1.0000,  0.0000,  0.0000,  0.0000,  2.0000,
          0.6800,  0.6364,  0.8900,  0.1343],
        [ 1.0000,  1.0000,  3.0000,  3.0000,  0.0000,  0.0000,  0.0000,  1.0000,
          0.2400,  0.2

Input and output sizes

In [202]:
input_size = len(input_cols)
output_size = len(output_cols)
hidden_size1 = 24
hidden_size2 = 12
input_size, output_size

(12, 1)

Model definition

In [203]:
class ScooterModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear1 = nn.Linear(input_size, hidden_size1)
        self.act_fn1 = nn.Sigmoid()
        self.linear2 = nn.Linear(hidden_size1, hidden_size2)
        self.act_fn2 = nn.ReLU()
        self.linear3 = nn.Linear(hidden_size2, output_size)

    def forward(self, xb):
        out = self.linear1(xb)
        out = self.act_fn1(out)
        out = self.linear2(out)
        out = self.act_fn2(out)
        out = self.linear3(out)

        return out

    def training_step(self, batch):
        inputs, targets = batch
        inputs, targets = inputs.to(device), targets.to(device)
        # Generate predictions
        out = self(inputs)  # Forward called
        # Calculate loss
        loss = F.l1_loss(out, targets)
        return loss

    def validation_step(self, batch):
        inputs, targets = batch
        inputs, targets = inputs.to(device), targets.to(device)
        # Generate predictions
        out = self(inputs)
        # Calculate loss
        loss = F.l1_loss(out, targets)
        return {'val_loss': loss.detach()}

    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()
        return {'val_loss': epoch_loss.item()}

    def epoch_end(self, epoch, result, num_epochs):
        # Print result every 100th epoch
        if (epoch + 1) % 1 == 0 or epoch == num_epochs - 1:
            print(f'Epoch [{epoch + 1}] >> val_loss: {result["val_loss"]:.4f}')


In [204]:
model = ScooterModel()
model.to(device)
model.train()

ScooterModel(
  (linear1): Linear(in_features=12, out_features=24, bias=True)
  (act_fn1): Sigmoid()
  (linear2): Linear(in_features=24, out_features=12, bias=True)
  (act_fn2): ReLU()
  (linear3): Linear(in_features=12, out_features=1, bias=True)
)

In [205]:
def validate(model, val_loader):
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.SGD):
    history = []
    optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs):
        # Training Phase
        for batch in train_loader:
            loss = model.training_step(batch)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        # Validation phase
        result = validate(model, val_loader)
        model.epoch_end(epoch, result, epochs)
        history.append(result)
    return history

Train model

In [206]:
epochs = 1500
lr = 1e-3
history = fit(epochs, lr, model, train_loader, val_loader)

Epoch [1] >> val_loss: 190.5885
Epoch [2] >> val_loss: 170.4632
Epoch [3] >> val_loss: 129.9832
Epoch [4] >> val_loss: 116.6118
Epoch [5] >> val_loss: 113.4118
Epoch [6] >> val_loss: 111.9218
Epoch [7] >> val_loss: 110.6564
Epoch [8] >> val_loss: 109.5857
Epoch [9] >> val_loss: 108.5644
Epoch [10] >> val_loss: 107.6385
Epoch [11] >> val_loss: 106.6981
Epoch [12] >> val_loss: 105.7569
Epoch [13] >> val_loss: 104.8782
Epoch [14] >> val_loss: 104.3581
Epoch [15] >> val_loss: 103.3065
Epoch [16] >> val_loss: 102.4340
Epoch [17] >> val_loss: 101.4543
Epoch [18] >> val_loss: 100.4528
Epoch [19] >> val_loss: 99.5510
Epoch [20] >> val_loss: 98.7610
Epoch [21] >> val_loss: 97.7206
Epoch [22] >> val_loss: 97.1262
Epoch [23] >> val_loss: 96.3970
Epoch [24] >> val_loss: 96.2055
Epoch [25] >> val_loss: 94.0186
Epoch [26] >> val_loss: 93.5416
Epoch [27] >> val_loss: 92.1328
Epoch [28] >> val_loss: 91.5063
Epoch [29] >> val_loss: 90.6540
Epoch [30] >> val_loss: 90.5100
Epoch [31] >> val_loss: 89.5473

In [207]:
state_dict = model.state_dict()
state_dict

OrderedDict([('linear1.weight',
              tensor([[ 3.2895e-01, -1.6357e+00, -2.3684e+00,  7.8153e-02,  4.8248e+00,
                        1.3472e-01,  9.6757e-01,  3.7392e-01, -8.3948e+00, -6.6449e+00,
                        5.8282e-01,  2.5847e+00],
                      [-6.0197e-01,  5.2537e-01, -3.3140e-04,  2.8365e+00,  3.2720e-01,
                       -3.2082e+00,  1.7723e+00,  2.6871e-01, -2.6532e-01, -4.9566e-01,
                        6.9863e-01, -2.6898e-01],
                      [-4.0512e-01,  3.4261e+01,  5.2678e-02, -2.5116e+00,  3.0468e+00,
                       -1.0589e-01,  1.4745e+00, -1.1441e-01,  1.4619e-01,  1.0690e+00,
                       -4.9231e-01,  2.1755e-01],
                      [ 6.8193e-02,  3.6987e-01, -2.2754e-02,  4.8296e+00, -1.3057e-01,
                       -8.5392e-02, -2.4458e+01, -8.9532e-02,  2.2887e+00, -5.9552e-01,
                       -3.7468e-01, -3.2393e-01],
                      [-9.4536e-02, -2.6538e+00, -4.1978e-02,  1

In [208]:
torch.save(state_dict, 'sol_model.tar')

In [211]:
model.eval()
df = pd.read_csv('evaluation_data.csv')
df1 = df.drop(['dteday'], axis=1)

test_tensor = torch.from_numpy(df1.values.astype(float)).to(device).float()
final_pred = model(test_tensor)
predictions = final_pred.cpu().detach().numpy()
pd.DataFrame(predictions).to_csv("evaluation_data.csv",header=False)
